# Exercise 1

## Setup

First, let us import some packages that are commonly used in this notebook. Please make sure your packages have been upgraded to the latest version if they do not work. This can be done with pip.

pip install \-\-upgrade [package name] (for Python 2)
<br>
pip3 install \-\-upgrade [package name] (for Python 3)

Please note Python 3 is recommended for this course.

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.datasets import load_boston

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

## End-to-end machine learning project
In this section, we perform regression tasks for predicting housing prices with regression algorithms implemented by _scikit-learn_. _scikit-learn_ is a popular machine learning library, which features various algorithms on classification, regression and clustering. Most algorithms introduced in this course will be implemented in _scikit-learn_.

### Load Boston housing price dataset
Boston housing price dataset contains 506 samples, each of which consists of 13 features and 1 label of housing price values. The features are:

1. CRIM: per capita crime rate by town.
2. ZN: proportion of residential land zoned for lots over 25,000 sq.ft.
3. INDUS: proportion of non-retail business acres per town.
4. CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
5. NOX: nitrogen oxides concentration (parts per 10 million).
6. RM: average number of rooms per dwelling.
7. AGE: proportion of owner-occupied units built prior to 1940.
8. DIS: weighted mean of distances to five Boston employment centres.
9. RAD: index of accessibility to radial highways.
10. TAX: full-value property-tax rate per $10,000.
11. PTRATIO: pupil-teacher ratio by town.
12. B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.
13. LSTAT: lower status of the population (percent).

This mini project will let you solve a simple regression problem.

In [2]:
boston = load_boston()
X = boston['data']
y = boston['target']

The full description of Boston housing price dataset can be viewed by the following function.

### Dataset preprocessing
In this section, we will take a look at different features. Please note that these steps are not required in predicting Boston housing prices, but just give you some insights of various types of data.

#### Missing data items

Question:

Try other strategies (for example, replacing missing values with median or most frequent value).

In [ ]:
# randomly replace some entries with nan
np.random.seed(42)

X_new = X.copy()
mask = np.random.randint(0, 2, size=X.shape).astype(np.bool)
X_new[mask] = np.nan

In [5]:
# solution
# median
imp = Imputer(strategy='median')
X_replace_with_median = imp.fit_transform(X_new)

imp = Imputer(strategy='most_frequent')
X_replace_with_freq = imp.fit_transform(X_new)

### Training and test data
The dataset is generally split into two parts: training set and test set. We use the training set for training a model, and apply the trained model to the test set, in order to evaluate the performance of our model.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Standardisation of dataset
Standardisation of a dataset is a very common technique in machine learning. This process removes the mean and scales each feature column to unit variance.

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Regression
This section solves the regression problem with different methods. We also compute mean squared error (MSE) and mean absolute error (MAE) for future use.

### Linear regression

Question:

Perform linear regression, but with one feature at a time. Compute the MSEs and MAEs.

In [1]:
# solution

n_sample, n_feature = X_train.shape
print(np.reshape(X_train[:, 0], [X_train.shape[0], 1]))
mse_lr_per_feature = []
mae_lr_per_feature = []
for counter in range(X_train.shape[1]):
    lr = LinearRegression()
    lr.fit(np.reshape(X_train[:, counter], [X_train.shape[0], 1]), y_train)   
    y_lr = lr.predict(np.reshape(X_test[:, counter], [X_test.shape[0], 1]))
    
    
    mse_lr_per_feature.append(mean_squared_error(y_test, y_lr))
    mae_lr_per_feature.append(mean_absolute_error(y_test, y_lr))

errors = pd.DataFrame.from_dict({'MAE': mae_lr_per_feature,
                                 'MSE': mse_lr_per_feature},
                                 orient='index', columns=boston['feature_names'])
print(errors)

NameError: name 'X_train' is not defined

### K nearest neighbour (KNN) regression

Question:

Try other numbers and compare their MSEs and MAEs.

In [23]:
# solution

mse_neigh_other_num = []
mae_neigh_other_num = []
for n_neighbors in range(4, 7):
    neigh = KNeighborsRegressor(n_neighbors=n_neighbors)
    neigh.fit(X_train, y_train)
    
    y_neigh = neigh.predict(X_test)
    
    mse_neigh_other_num.append(mean_squared_error(y_test, y_neigh))
    mae_neigh_other_num.append(mean_absolute_error(y_test, y_neigh))


errors = pd.DataFrame.from_dict({'MAE': mae_neigh_other_num,
                                 'MSE': mse_neigh_other_num},
                                 orient='index', columns=range(4, 7))
errors

,4,5,6
MSE,19.963174,20.623929,22.407568
MAE,2.717647,2.592157,2.698529


### Gradient boosting regression

Question:

Try different types of loss functions as well as parameters. Compare the performances with MSEs and MAEs.

In [26]:
# solution

losses = ['ls', 'quantile']
alphas = np.linspace(0.7, 0.9, 5)

mse_gb_other_param = np.empty([len(losses), len(alphas)])
mae_gb_other_param = np.empty([len(losses), len(alphas)])
for loss_counter, loss in enumerate(losses):
    for alpha_counter, alpha in enumerate(alphas):
        gb = GradientBoostingRegressor(loss=loss, alpha=alpha,
                                        n_estimators=250, max_depth=3,
                                        learning_rate=.1, min_samples_leaf=9,
                                        min_samples_split=9)
        gb.fit(X_train, y_train)
        y_gb = gb.predict(X_test)
        
        mse_gb_other_param[loss_counter, alpha_counter] = \
            mean_squared_error(y_test, y_gb)
        mae_gb_other_param[loss_counter, alpha_counter] = \
            mean_absolute_error(y_test, y_gb)

In [27]:
# mse
pd.DataFrame(data=mse_gb_other_param,
             index=losses, columns=alphas)

,0.7,0.75,0.8,0.85,0.9
ls,7.277475,7.277475,7.277475,7.277475,7.277475
quantile,8.936678,10.563187,10.405786,14.251271,16.502917


In [28]:
# mae
pd.DataFrame(data=mae_gb_other_param,
             index=losses, columns=alphas)

,0.7,0.75,0.8,0.85,0.9
ls,2.000848,2.000848,2.000848,2.000848,2.000848
quantile,2.159376,2.279549,2.221113,2.603658,3.040635


Question:

Do different parameters have effects on the accuracy of our predictions? Could you come up with a method to automatically choose the value of parameters?

In [29]:
# solution
# Cross validation (which will be introduced in future lectures)